In [ ]:
#библиотеки

import numpy as np #для вычислений

In [ ]:
#инициализация глобальных переменных

num_cars = 7  #количество машин
num_companies = 4  #количество предприятий
probabilities = np.array([
    [1.0, 0.9, 0.7, 0.6, 0.6, 0.5, 0.4, 0.2],  #предприятие 1
    [1.0, 0.9, 0.8, 0.6, 0.5, 0.5, 0.3, 0.2],  #предприятие 2
    [1.0, 1.0, 0.9, 0.9, 0.9, 0.7, 0.7, 0.1],  #предприятие 3
    [1.0, 0.9, 0.8, 0.4, 0.2, 0.2, 0.2, 0.0]   #предприятие 4
]) #вероятности выполнения заказов (0-7 машин) предприятиями

In [ ]:
#функции
'''
функция solution позволяет пользователю выбрать формат данных: предопределённые из условия или введенные вручную,
    а затем вызывает функцию решения задачи
  вход: -
  выход: выводит решение
'''
def solution():
  print("Выберите способ ввода данных:\n1. Предопределённые значения\n2. Пользовательский ввод")
  choice = int(input("Введите 1 или 2: "))

  #проверка корректности выбора
  while choice not in {1, 2}:
    choice = int(input("Ошибка! Введите 1 или 2: "))

  if choice == 2:#проверка корректности выбора
    while choice not in {1, 2}:
      choice = int(input("Ошибка! Введите 1 или 2: "))

  if choice == 2:
    num_companies_2, num_cars_2, probabilities_2 = data_input()
    dynamic_prog(num_companies_2, num_cars_2, probabilities_2)

  else:
    dynamic_prog(num_companies, num_cars, probabilities)

'''
функция data_input для ввода данных пользователем:
  вход: -
  выход: возвращает введенные значения num_cars, num_companies, probabilities
'''
def data_input():
    #пользовательский ввод
    num_cars, num_companies, probabilities = 0, 0, []
    print("Введите данные:")
    while num_cars <= 0:
      num_cars = int(input("Количество машин = "))
    while num_companies <= 0:
      num_companies = int(input("Количество предприятий = "))

    for i in range(num_companies):
        while True:
            print(f"Введите вероятности для предприятия {i + 1}:")
            row_input = input()
            row = row_input.split()

            if len(row) != num_cars + 1:
                print(f"Ошибка! нужно ввести ровно {num_cars + 1} чисел.")
                continue

            row = [float(value) for value in row]
            if all(0 <= p <= 1 for p in row):
                probabilities.append(row)
                break
            else:
                print("Ошибка: все вероятности должны быть от 0 до 1.")

    probabilities = np.array(probabilities)
    return num_cars, num_companies, probabilities


def F_k_step(k, num_companies, num_cars, probabilities):
    """
    функция F_k_step для вычисления условно оптимального выигрыша на k-ом шаге
                (т.е. вычисления максимальной вероятности выполнения заказа,
                распределённого на k,k+1,...,num_companies предприятиях)
                из уравнений Беллмана:
                F_k(X_k-1) = max(u_k){W_k(X_k-1, u_k)} для k = num_companies
                F_k(X_k-1) = max(u_k){W_k(X_k-1, u_k) * F_k+1(X_k-1 - u_k)} для k от 1 до num_companies
  вход: k - номер шага
        num_companies - количество предприятий
        num_cars - количество машин
        probabilities - массив вероятностей выполнения заказов (0-7 машин) предприятиями

  выход: массив F_k_arr вида:
         [[F_k(X_k-1 = 0), u_k11, u_k12, ...], ..., [F_k(X_k-1 = num_cars), u_kn1, u_kn2, ...]],
         где u_k - условно оптимальные управления
    """
    #для вычисления F_k, где k - номер последнего предприятия
    if k == num_companies:
        return [[probabilities[k - 1][u_k], u_k] for u_k in range(num_cars + 1)]

    F_k_arr = []
    #находим F_k(X_k-1), где X_k-1 от 0 до num_cars,
    #т.е. вероятность того, что k-ое предприятие выполнит заказ на u_k машин, если после заказа на предыдущем предприятии осталось заказать X_k-1 машин
    for X_k_1 in range(num_cars + 1):
        temp_values = []

        # Перебираем все возможные значения u_k
        for u_k in range(X_k_1 + 1):
            next_step = F_k_step(k + 1, num_companies, num_cars, probabilities)
            prob = probabilities[k - 1][u_k] * next_step[X_k_1 - u_k][0]
            temp_values.append(prob)

        max_prob = max(temp_values)
        optimal_allocations = [u_k for u_k, value in enumerate(temp_values) if value == max_prob]
        F_k_arr.append([max_prob] + optimal_allocations)

    return F_k_arr


def dynamic_prog(num_companies, num_cars, probabilities):
    """
    функция dynamic_prog для решения задачи динамическим программированием
        (выводит все оптимальные решения задачи)
    вход: num_companies - количество предприятий
        num_cars - количество машин
        probabilities - массив вероятностей выполнения заказов (0-7 машин) предприятиями
    """
    F_arr = []

    # записываем выигрыши от последнего предприятия к первому
    for k in range(num_companies, 0, -1):
        F_arr.insert(0, F_k_step(k, num_companies, num_cars, probabilities))

    #выводим оптимальное распределение
    X_k_1 = num_cars
    u_k = int(F_arr[0][X_k_1][1])
    print(f"It should be ordered {u_k} on factory 1")

    for k in range(1, num_companies):
        X_k_1 -= u_k
        u_k = int(F_arr[k][X_k_1][1])
        print(f"It should be ordered {u_k}  on factory {k + 1}")

    print(f"Total probability is {F_arr[0][num_cars][0]}")


def F_k_step_v2(k, num_companies, num_cars, max_productions, probabilities):
    """
    Вычисляет условно оптимальный выигрыш для этапа k с учётом максимальной производительности заводов
    вход: k - номер шага
          num_companies - количество предприятий
          num_cars - количество машин
          max_productions - массив с максимальной производительностью каждого завода
          probabilities - массив вероятностей выполнения заказов (0-7 машин) предприятиями

    выход: массив F_k_arr вида:
           [[F_k(X_k-1 = 0), u_k11, u_k12, ...], ..., [F_k(X_k-1 = num_cars), u_kn1, u_kn2, ...]],
           где u_k - условно оптимальные управления.
    """
    if k == num_companies:
        #для последнего предприятия
        return [[probabilities[k - 1][u_k], u_k] for u_k in range(min(num_cars, max_productions[k - 1]) + 1)]

    F_k_arr = []
    #рассчитываем F_k для каждого значения X_k-1
    for X_k_1 in range(num_cars + 1):
        temp_values = []

        #перебираем все возможные значения u_k, ограниченные производительностью предприятия
        for u_k in range(min(X_k_1, max_productions[k - 1]) + 1):
            next_step = F_k_step_v2(k + 1, num_companies, num_cars, max_productions, probabilities)
            prob = probabilities[k - 1][u_k] * next_step[X_k_1 - u_k][0]
            temp_values.append(prob)

        max_prob = max(temp_values)
        optimal_allocations = [u_k for u_k, value in enumerate(temp_values) if value == max_prob]
        F_k_arr.append([max_prob] + optimal_allocations)

    return F_k_arr


def dynamic_prog_v2(num_companies, num_cars, max_productions, probabilities):
    """
    функция dynamic_prog_v2 для решения задачи динамическим программированием с учётом максимальной производительности заводов
    вход: num_companies - количество предприятий
        num_cars - количество машин
        max_productions - массив с максимальной производительностью каждого завода
        probabilities - массив вероятностей выполнения заказов (0-7 машин) предприятиями
    """
    F_arr = []
    #выигрыши от последнего предприятия к первому
    for k in range(num_companies, 0, -1):
        F_arr.insert(0, F_k_step_v2(k, num_companies, num_cars, max_productions, probabilities))

    #выводим оптимальное распределение
    X_k_1 = num_cars
    u_k = int(F_arr[0][X_k_1][1])
    print(f"It should be ordered {u_k} on factory 1")

    for k in range(1, num_companies):
        X_k_1 -= u_k
        u_k = int(F_arr[k][X_k_1][1])
        print(f"It should be ordered {u_k}  on factory {k + 1}")

    print(f"Total probability is {F_arr[0][num_cars][0]}")

In [ ]:
solution()

Выберите способ ввода данных:
1. Предопределённые значения
2. Пользовательский ввод
Введите 1 или 2: 1
It should be ordered 1 on factory 1
It should be ordered 1  on factory 2
It should be ordered 4  on factory 3
It should be ordered 1  on factory 4
Total probability is 0.6561000000000001


In [ ]:
max_productions = [7,7,3,7]
dynamic_prog_v2(num_companies, num_cars, max_productions, probabilities)

It should be ordered 1 on factory 1
It should be ordered 1  on factory 2
It should be ordered 3  on factory 3
It should be ordered 2  on factory 4
Total probability is 0.5832000000000002


In [ ]:
max_productions = [10,10,1,10]
dynamic_prog_v2(num_companies, num_cars, max_productions, probabilities)

It should be ordered 4 on factory 1
It should be ordered 1  on factory 2
It should be ordered 1  on factory 3
It should be ordered 1  on factory 4
Total probability is 0.486
